In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pytorch-pretrained-bert pytorch-nlp

     |████████████████████████████████| 133kB 5.2MB/s 
     |████████████████████████████████| 92kB 4.6MB/s 
     |████████████████████████████████| 133kB 8.2MB/s 
     |████████████████████████████████| 71kB 6.4MB/s 
     |████████████████████████████████| 6.7MB 12.7MB/s 
ERROR: botocore 1.19.9 has requirement urllib3<1.26,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [1]:
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
from statistics import mode

import warnings

warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", DeprecationWarning)

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla V100-SXM2-16GB'

In [7]:
def train_validate_split(df,seed=42,validate_percent = 0.1):
  train, validate = train_test_split(df, test_size=validate_percent, stratify=df['label'])
  return train, validate

def train_validate_test_split(df,seed, train_percent=.8, validate_percent=.125):
  train, test = train_test_split(df, train_size=train_percent, stratify=df['label'])
  train, validate = train_test_split(df, test_size=validate_percent, stratify=df['label'])
  return train, validate, test

def sample_data(df,sample,seed):
    X_train, _, y_train, _ = train_test_split( df['tweet'], df['label'], train_size=sample, random_state=seed, stratify=df['label'])
    return pd.concat([X_train,y_train], axis = 1 )

def tokenize_data(df):
    sentences = ["[CLS] " + query + " [SEP]" for query in df['tweet']]
    # Tokenize with multilingual BERT tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    MAX_LEN = 128

    # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    # Create attention masks
    attention_masks = []
    # Create a mask of 1s for each token followed by 0s for padding
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    return input_ids, attention_masks

def Data_Loader(inputs_ids, attention_masks, df, batch_size=16):
    data = TensorDataset(torch.LongTensor(inputs_ids), torch.LongTensor(attention_masks), torch.LongTensor(df['label'].values))
    dataloader = DataLoader(data, sampler=RandomSampler(data), batch_size=batch_size)
    return dataloader

In [3]:
def model_train(model, train_dataloader, validation_dataloader):
    # Store our loss and accuracy for plotting
    train_loss_set = []
    # BERT training loop
    epochs = 3
    for _ in trange(epochs, desc="Epoch"):  
        # Set our model to training mode
        model.train()
        # Tracking variables
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        # Train the data for one epoch
        for step, batch in enumerate(train_dataloader):
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()
            # Forward pass
            loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            train_loss_set.append(loss.item())    
            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()
            # Update tracking variables
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1
        print("Train loss: {}".format(tr_loss/nb_tr_steps))

        ## VALIDATION

        # Put model in evaluation mode
        model.eval()
        # Tracking variables 
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        # Evaluate data for one epoch
        for batch in validation_dataloader:
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            # Telling the model not to compute or store gradients, saving memory and speeding up validation
            with torch.no_grad():
                # Forward pass, calculate logit predictions
                logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)    
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            tmp_eval_accuracy = flat_accuracy(logits, label_ids)    
            eval_accuracy += tmp_eval_accuracy
            nb_eval_steps += 1
        validation_accuracy = (eval_accuracy/nb_eval_steps)
        print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    return validation_accuracy

In [4]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def model_test(model,prediction_dataloader):
    model.eval()
    # Tracking variables 
    predictions , true_labels = [], []
# Predict 
    for batch in prediction_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up prediction
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions+=list(np.argmax(logits, axis=1).flatten())
        true_labels+=list(label_ids.flatten())

    test_f1_score = f1_score(true_labels, predictions, average= 'macro')
    print("Macro F1 Score:",test_f1_score)
    test_accuracy_score = accuracy_score(true_labels, predictions)
    print("Accuracy score:", test_accuracy_score, "\n")
    print("="*100)

In [5]:
def model_initialise(use_saved_model=False):
  # Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
  model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=2).cuda()

  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'gamma', 'beta']
  optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}]

  optimizer = BertAdam(optimizer_grouped_parameters, lr=2e-5, warmup=.1)
  
  if(use_saved_model==True):
    checkpoint = torch.load('/content/drive/My Drive/Multilingual models/mBERT.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

  return model, optimizer

In [8]:
languages = {'fr_ar':'English'}#'en_fr':'Arabic','fr_ar':'English','en_ar':'French'}
directory = './'
drive_directory = '/content/drive/My Drive/Multilingual models'
for lang1, lang2 in languages.items():
	df = pd.read_csv(os.path.join(directory, lang1+'.csv'))
	train_df, validation_df = train_validate_split(df)
	train_input_ids, train_attention_masks = tokenize_data(train_df)
	train_dataloader = Data_Loader(train_input_ids, train_attention_masks, train_df)
	validation_input_ids, validation_attention_masks = tokenize_data(validation_df)
	validation_dataloader = Data_Loader(validation_input_ids, validation_attention_masks, validation_df)
	model, optimizer = model_initialise()
	validation_accuracy = model_train(model, train_dataloader, validation_dataloader)
	fname = 'mBERT.pth'
	path = os.path.join(drive_directory, fname)
	torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, path)
	test_df = pd.read_csv(os.path.join(directory, lang2+'.csv'))
	test_input_ids, test_attention_masks = tokenize_data(test_df)
	test_dataloader = Data_Loader(test_input_ids, test_attention_masks, test_df)
	print("\nZero Shot Model Summary:")
	print('Language:', lang2,'\n')
	model_test(model, test_dataloader)
	sample_sizes = [16, 32, 64, 128, 256]
	for sample in sample_sizes:
		df = pd.read_csv(os.path.join(directory, lang2+'.csv'))
		#best_validation_score = 0
		seeds = [2018, 2019, 2020, 2021, 2022]
		for seed in seeds:
			train_df, validation_df, test_df = train_validate_test_split(df,seed)
			train_len = len(train_df)
			if sample==256 and seed==2022:
			    sample_sizes.append(train_len)
			if sample == train_len and seed == 2022:
			    sample_sizes.remove(train_len)
			model, optimizer = model_initialise(use_saved_model=True)
			if (sample != train_len):
				train_df_sample = sample_data(train_df,sample,seed)
				train_input_ids, train_attention_masks = tokenize_data(train_df_sample)
				train_dataloader = Data_Loader(train_input_ids, train_attention_masks, train_df_sample)
			else:
				train_input_ids, train_attention_masks = tokenize_data(train_df)
				train_dataloader = Data_Loader(train_input_ids, train_attention_masks, train_df)

			validation_input_ids, validation_attention_masks = tokenize_data(validation_df)
			validation_dataloader = Data_Loader(validation_input_ids, validation_attention_masks, validation_df)
			print("\nModel Summary:")
			print('Language:', lang2)
			print('Sample Size:', sample)
			print('Seed value:', seed)
			validation_accuracy = model_train(model, train_dataloader, validation_dataloader)
			#if(validation_accuracy > best_validation_score):
			#	best_validation_score = validation_accuracy
			#	print('Language:', lang)
			#	print("The Best Seed for the sample ",sample, " is : ",seed)
			#	fname = "mBERT_"+str(lang)+"_"+str(sample)+'.pth'
			#	path = os.path.join(drive_directory, fname)
			#	torch.save(model.state_dict(), path)
			test_input_ids, test_attention_masks = tokenize_data(test_df)
			test_dataloader = Data_Loader(test_input_ids, test_attention_masks, test_df)
			model_test(model, test_dataloader)

t_total value of -1 results in schedule not being applied
Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Train loss: 0.5278833996646012


Epoch:  33%|███▎      | 1/3 [01:30<03:01, 90.82s/it]

Validation Accuracy: 0.7695585664335663
Train loss: 0.5110972972987573


Epoch:  67%|██████▋   | 2/3 [03:01<01:30, 90.79s/it]

...

Epoch:  67%|██████▋   | 2/3 [43:45<21:52, 1312.68s/it]

Validation Accuracy: 0.8069698808473081
Train loss: 0.49193884230739243


Epoch: 100%|██████████| 3/3 [1:05:45<00:00, 1315.04s/it]

Validation Accuracy: 0.8069354037952339


Macro F1 Score: 0.4465907301453992
Accuracy score: 0.80698093521768 



In [ ]:
#RandomSampler
#stratifysplit
#Seed as a parameter

In [ ]:
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]Train loss: 0.5053966010673137
# Epoch:  33%|███▎      | 1/3 [02:11<04:22, 131.32s/it]Validation Accuracy: 0.805506993006993
# Train loss: 0.3801150870285457
# Epoch:  67%|██████▋   | 2/3 [04:22<02:11, 131.42s/it]Validation Accuracy: 0.8296547202797203
# Train loss: 0.31789775152183786
# Epoch: 100%|██████████| 3/3 [06:34<00:00, 131.53s/it]Validation Accuracy: 0.8456075174825174

# Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (552 > 512). Running this sequence through BERT will result in indexing errors

# Zero Shot Model Summary:
# Language: English 

# Macro F1 Score: 0.47023059082806423
# Accuracy score: 0.7881262686623793 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 16
# Seed value: 2018
# Train loss: 0.4717906415462494
# Epoch:  33%|███▎      | 1/3 [01:00<02:01, 60.73s/it]Validation Accuracy: 0.7908966791703442
# Train loss: 0.4479028582572937
# Epoch:  67%|██████▋   | 2/3 [02:01<01:00, 60.72s/it]Validation Accuracy: 0.7960889232127096
# Train loss: 0.45949700474739075
# Epoch: 100%|██████████| 3/3 [03:02<00:00, 60.72s/it]Validation Accuracy: 0.7994263018534864

# Macro F1 Score: 0.4611711839858038
# Accuracy score: 0.7988712890069241 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 16
# Seed value: 2019
# Train loss: 0.4277941882610321
# Epoch:  33%|███▎      | 1/3 [01:00<02:01, 60.72s/it]Validation Accuracy: 0.7877730582524272
# Train loss: 0.40369072556495667
# Epoch:  67%|██████▋   | 2/3 [02:01<01:00, 60.72s/it]Validation Accuracy: 0.7924481465136806
# Train loss: 0.3882438838481903
# Epoch: 100%|██████████| 3/3 [03:02<00:00, 60.72s/it]Validation Accuracy: 0.7968474183583407

# Macro F1 Score: 0.4586245473515313
# Accuracy score: 0.7962154984349805 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 16
# Seed value: 2020
# Train loss: 0.4772171080112457
# Epoch:  33%|███▎      | 1/3 [01:00<02:01, 60.73s/it]Validation Accuracy: 0.7904760591350397
# Train loss: 0.47575828433036804
# Epoch:  67%|██████▋   | 2/3 [02:01<01:00, 60.72s/it]Validation Accuracy: 0.7950615070609003
# Train loss: 0.432832270860672
# Epoch: 100%|██████████| 3/3 [03:02<00:00, 60.68s/it]Validation Accuracy: 0.7995021513680495

# Macro F1 Score: 0.4613231249135639
# Accuracy score: 0.7981599165322963 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 16
# Seed value: 2021
# Train loss: 0.5424805283546448
# Epoch:  33%|███▎      | 1/3 [01:00<02:01, 60.65s/it]Validation Accuracy: 0.7881247241835835
# Train loss: 0.5007271766662598
# Epoch:  67%|██████▋   | 2/3 [02:01<01:00, 60.66s/it]Validation Accuracy: 0.7938065423654016
# Train loss: 0.4823650121688843
# Epoch: 100%|██████████| 3/3 [03:01<00:00, 60.66s/it]Validation Accuracy: 0.7971094439541042

# Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (552 > 512). Running this sequence through BERT will result in indexing errors
# Macro F1 Score: 0.4612569418406465
# Accuracy score: 0.7985393151854311 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 16
# Seed value: 2022
# Train loss: 0.555505633354187
# Epoch:  33%|███▎      | 1/3 [01:00<02:01, 60.66s/it]Validation Accuracy: 0.79407546337158
# Train loss: 0.5260133147239685
# Epoch:  67%|██████▋   | 2/3 [02:01<01:00, 60.66s/it]Validation Accuracy: 0.7999917255075023
# Train loss: 0.4799751043319702
# Epoch: 100%|██████████| 3/3 [03:01<00:00, 60.65s/it]Validation Accuracy: 0.802680935569285

# Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (552 > 512). Running this sequence through BERT will result in indexing errors
# Macro F1 Score: 0.4568895209157646
# Accuracy score: 0.8033766480128995 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 32
# Seed value: 2018
# Train loss: 0.5429712533950806
# Epoch:  33%|███▎      | 1/3 [01:01<02:02, 61.01s/it]Validation Accuracy: 0.7952476831421006
# Train loss: 0.4931335747241974
# Epoch:  67%|██████▋   | 2/3 [02:01<01:00, 60.99s/it]Validation Accuracy: 0.8045495917917034
# Train loss: 0.42926056683063507
# Epoch: 100%|██████████| 3/3 [03:02<00:00, 60.97s/it]Validation Accuracy: 0.8059079876434246

# Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (552 > 512). Running this sequence through BERT will result in indexing errors
# Macro F1 Score: 0.45026048694246656
# Accuracy score: 0.8064118372379778 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 32
# Seed value: 2019
# Train loss: 0.5250117778778076
# Epoch:  33%|███▎      | 1/3 [01:00<02:01, 60.94s/it]Validation Accuracy: 0.7979782656663724
# Train loss: 0.48672865331172943
# Epoch:  67%|██████▋   | 2/3 [02:01<01:00, 60.94s/it]Validation Accuracy: 0.8054184135039718
# Train loss: 0.4289259910583496
# Epoch: 100%|██████████| 3/3 [03:02<00:00, 60.94s/it]Validation Accuracy: 0.8070457303618712

# Macro F1 Score: 0.45187559007907296
# Accuracy score: 0.8062695627430523 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 32
# Seed value: 2020
# Train loss: 0.5739918947219849
# Epoch:  33%|███▎      | 1/3 [01:00<02:01, 60.99s/it]Validation Accuracy: 0.7974542144748455
# Train loss: 0.5054817497730255
# Epoch:  67%|██████▋   | 2/3 [02:01<01:00, 60.98s/it]Validation Accuracy: 0.8047012908208296
# Train loss: 0.4913571774959564
# Epoch: 100%|██████████| 3/3 [03:02<00:00, 60.98s/it]Validation Accuracy: 0.8063975617828772

# Macro F1 Score: 0.44845046159494756
# Accuracy score: 0.8059850137532012 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 32
# Seed value: 2021
# Train loss: 0.5247595608234406
# Epoch:  33%|███▎      | 1/3 [01:01<02:02, 61.05s/it]Validation Accuracy: 0.7959717012356576
# Train loss: 0.5292621850967407
# Epoch:  67%|██████▋   | 2/3 [02:02<01:01, 61.03s/it]Validation Accuracy: 0.8049150485436893
# Train loss: 0.4499519616365433
# Epoch: 100%|██████████| 3/3 [03:03<00:00, 61.01s/it]Validation Accuracy: 0.8062458627537511

# Macro F1 Score: 0.45173245921563127
# Accuracy score: 0.8058427392582757 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 32
# Seed value: 2022
# Train loss: 0.4845106452703476
# Epoch:  33%|███▎      | 1/3 [01:00<02:01, 60.98s/it]Validation Accuracy: 0.7985919571932921
# Train loss: 0.4618699625134468
# Epoch:  67%|██████▋   | 2/3 [02:01<01:00, 60.98s/it]Validation Accuracy: 0.8035152802294793
# Train loss: 0.39411772787570953
# Epoch: 100%|██████████| 3/3 [03:02<00:00, 60.98s/it]Validation Accuracy: 0.8060596866725508

# Macro F1 Score: 0.45311200307443755
# Accuracy score: 0.8057478895949919 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 64
# Seed value: 2018
# Train loss: 0.5106881260871887
# Epoch:  33%|███▎      | 1/3 [01:01<02:03, 61.63s/it]Validation Accuracy: 0.803329104148279
# Train loss: 0.47872521728277206
# Epoch:  67%|██████▋   | 2/3 [02:03<01:01, 61.63s/it]Validation Accuracy: 0.8067354368932039
# Train loss: 0.4434658959507942
# Epoch: 100%|██████████| 3/3 [03:04<00:00, 61.63s/it]Validation Accuracy: 0.8066733781994704

# Macro F1 Score: 0.4470174486163689
# Accuracy score: 0.8067912358911126 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 64
# Seed value: 2019
# Train loss: 0.5158382132649422
# Epoch:  33%|███▎      | 1/3 [01:01<02:03, 61.75s/it]Validation Accuracy: 0.8056459620476611
# Train loss: 0.46764667704701424
# Epoch:  67%|██████▋   | 2/3 [02:03<01:01, 61.72s/it]Validation Accuracy: 0.8066251103265666
# Train loss: 0.4278232492506504
# Epoch: 100%|██████████| 3/3 [03:05<00:00, 61.72s/it]Validation Accuracy: 0.8066251103265666

# Macro F1 Score: 0.4470174486163689
# Accuracy score: 0.8067912358911126 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (552 > 512). Running this sequence through BERT will result in indexing errors
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 64
# Seed value: 2020
# Train loss: 0.5185684934258461
# Epoch:  33%|███▎      | 1/3 [01:01<02:03, 61.65s/it]Validation Accuracy: 0.8045358009708737
# Train loss: 0.457609623670578
# Epoch:  67%|██████▋   | 2/3 [02:03<01:01, 61.64s/it]Validation Accuracy: 0.8066251103265666
# Train loss: 0.4351537898182869
# Epoch: 100%|██████████| 3/3 [03:04<00:00, 61.64s/it]Validation Accuracy: 0.8065561562224184

# Macro F1 Score: 0.44792463669633753
# Accuracy score: 0.8066015365645451 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 64
# Seed value: 2021
# Train loss: 0.577980637550354
# Epoch:  33%|███▎      | 1/3 [01:01<02:03, 61.68s/it]Validation Accuracy: 0.8055976941747572
# Train loss: 0.49318312108516693
# Epoch:  67%|██████▋   | 2/3 [02:03<01:01, 61.68s/it]Validation Accuracy: 0.8069698808473081
# Train loss: 0.4743781164288521
# Epoch: 100%|██████████| 3/3 [03:05<00:00, 61.68s/it]Validation Accuracy: 0.8070871028243601

# Macro F1 Score: 0.4465907301453992
# Accuracy score: 0.80698093521768 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 64
# Seed value: 2022
# Train loss: 0.48800259828567505
# Epoch:  33%|███▎      | 1/3 [01:01<02:03, 61.65s/it]Validation Accuracy: 0.8031360326566637
# Train loss: 0.42422161996364594
# Epoch:  67%|██████▋   | 2/3 [02:03<01:01, 61.65s/it]Validation Accuracy: 0.8054597859664607
# Train loss: 0.3483731970191002
# Epoch: 100%|██████████| 3/3 [03:04<00:00, 61.64s/it]Validation Accuracy: 0.802005185348632

# Macro F1 Score: 0.46715937467548285
# Accuracy score: 0.8024281513800626 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (552 > 512). Running this sequence through BERT will result in indexing errors
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 128
# Seed value: 2018
# Train loss: 0.5064125880599022
# Epoch:  33%|███▎      | 1/3 [01:02<02:05, 62.97s/it]Validation Accuracy: 0.806928508384819
# Train loss: 0.46234886907041073
# Epoch:  67%|██████▋   | 2/3 [02:05<01:02, 62.98s/it]Validation Accuracy: 0.8072663834951457
# Train loss: 0.38400549441576004
# Epoch: 100%|██████████| 3/3 [03:08<00:00, 62.98s/it]Validation Accuracy: 0.7741339364518977

# Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (552 > 512). Running this sequence through BERT will result in indexing errors
# Macro F1 Score: 0.5208495855072175
# Accuracy score: 0.7753011476809257 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 128
# Seed value: 2019
# Train loss: 0.5375264436006546
# Epoch:  33%|███▎      | 1/3 [01:02<02:05, 62.89s/it]Validation Accuracy: 0.8062458627537511
# Train loss: 0.4792347699403763
# Epoch:  67%|██████▋   | 2/3 [02:05<01:02, 62.89s/it]Validation Accuracy: 0.806252758164166
# Train loss: 0.45361604169011116
# Epoch: 100%|██████████| 3/3 [03:08<00:00, 62.88s/it]Validation Accuracy: 0.8066595873786407

# Macro F1 Score: 0.48304737862623026
# Accuracy score: 0.806648961396187 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 128
# Seed value: 2020
# Train loss: 0.49855519086122513
# Epoch:  33%|███▎      | 1/3 [01:02<02:05, 62.93s/it]Validation Accuracy: 0.8071560569285083
# Train loss: 0.43617353215813637
# Epoch:  67%|██████▋   | 2/3 [02:05<01:02, 62.94s/it]Validation Accuracy: 0.8070871028243601
# Train loss: 0.34183981269598007
# Epoch: 100%|██████████| 3/3 [03:08<00:00, 62.94s/it]Validation Accuracy: 0.7736719439541042

# Macro F1 Score: 0.5229453864162141
# Accuracy score: 0.7681399981030067 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 128
# Seed value: 2021
# Train loss: 0.5259784199297428
# Epoch:  33%|███▎      | 1/3 [01:02<02:05, 62.89s/it]Validation Accuracy: 0.8065078883495146
# Train loss: 0.4545721635222435
# Epoch:  67%|██████▋   | 2/3 [02:05<01:02, 62.89s/it]Validation Accuracy: 0.8040600176522507
# Train loss: 0.3946741446852684
# Epoch: 100%|██████████| 3/3 [03:08<00:00, 62.88s/it]Validation Accuracy: 0.791427625772286

# Macro F1 Score: 0.5110466136781615
# Accuracy score: 0.7923266622403491 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 128
# Seed value: 2022
# Train loss: 0.5177350677549839
# Epoch:  33%|███▎      | 1/3 [01:02<02:05, 62.91s/it]Validation Accuracy: 0.8067078552515446
# Train loss: 0.45896995440125465
# Epoch:  67%|██████▋   | 2/3 [02:05<01:02, 62.90s/it]Validation Accuracy: 0.8055356354810238
# Train loss: 0.4079135060310364
# Epoch: 100%|██████████| 3/3 [03:08<00:00, 62.90s/it]Validation Accuracy: 0.8035566526919683

# Macro F1 Score: 0.4682894723441526
# Accuracy score: 0.8046096936355875 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 256
# Seed value: 2018
# Train loss: 0.5006420547142625
# Epoch:  33%|███▎      | 1/3 [01:05<02:10, 65.46s/it]Validation Accuracy: 0.8070043578993822
# Train loss: 0.45667532552033663
# Epoch:  67%|██████▋   | 2/3 [02:10<01:05, 65.46s/it]Validation Accuracy: 0.8064803067078553
# Train loss: 0.36601202096790075
# Epoch: 100%|██████████| 3/3 [03:16<00:00, 65.46s/it]Validation Accuracy: 0.7878902802294793

# Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (552 > 512). Running this sequence through BERT will result in indexing errors
# Macro F1 Score: 0.542491189491239
# Accuracy score: 0.7892914730152708 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 256
# Seed value: 2019
# Train loss: 0.513310993090272
# Epoch:  33%|███▎      | 1/3 [01:05<02:10, 65.47s/it]Validation Accuracy: 0.8069698808473081
# Train loss: 0.4580772574990988
# Epoch:  67%|██████▋   | 2/3 [02:10<01:05, 65.46s/it]Validation Accuracy: 0.8070802074139453
# Train loss: 0.36801680736243725
# Epoch: 100%|██████████| 3/3 [03:16<00:00, 65.45s/it]Validation Accuracy: 0.7712102824360106

# Macro F1 Score: 0.554919454162027
# Accuracy score: 0.7745423503746561 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 256
# Seed value: 2020
# Train loss: 0.5113286022096872
# Epoch:  33%|███▎      | 1/3 [01:05<02:10, 65.47s/it]Validation Accuracy: 0.8070802074139453
# Train loss: 0.4641929557546973
# Epoch:  67%|██████▋   | 2/3 [02:10<01:05, 65.49s/it]Validation Accuracy: 0.8068595542806708
# Train loss: 0.41411323472857475
# Epoch: 100%|██████████| 3/3 [03:16<00:00, 65.51s/it]Validation Accuracy: 0.7585020410414829

# Macro F1 Score: 0.579405836377325
# Accuracy score: 0.7672863511334534 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 256
# Seed value: 2021
# Train loss: 0.4959809333086014
# Epoch:  33%|███▎      | 1/3 [01:05<02:11, 65.55s/it]Validation Accuracy: 0.806252758164166
# Train loss: 0.420333806425333
# Epoch:  67%|██████▋   | 2/3 [02:11<01:05, 65.53s/it]Validation Accuracy: 0.8013156443071491
# Train loss: 0.3013382237404585
# Epoch: 100%|██████████| 3/3 [03:16<00:00, 65.50s/it]Validation Accuracy: 0.6843281112091791

# Macro F1 Score: 0.5832872949338735
# Accuracy score: 0.6851465427297733 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 256
# Seed value: 2022
# Train loss: 0.5066969618201256
# Epoch:  33%|███▎      | 1/3 [01:05<02:10, 65.48s/it]Validation Accuracy: 0.8072319064430714
# Train loss: 0.43413345515727997
# Epoch:  67%|██████▋   | 2/3 [02:10<01:05, 65.48s/it]Validation Accuracy: 0.7977851941747572
# Train loss: 0.35316015500575304
# Epoch: 100%|██████████| 3/3 [03:16<00:00, 65.47s/it]Validation Accuracy: 0.7250937775816417

# Macro F1 Score: 0.58093190407267
# Accuracy score: 0.7199563691548895 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (552 > 512). Running this sequence through BERT will result in indexing errors
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 92247
# Seed value: 2018
# Train loss: 0.3436069390579846
# Epoch:  33%|███▎      | 1/3 [31:45<1:03:31, 1905.66s/it]Validation Accuracy: 0.8644293358340689
# Train loss: 0.30104501576957876
# Epoch:  67%|██████▋   | 2/3 [1:03:41<31:48, 1908.73s/it]Validation Accuracy: 0.8702628530450132
# Train loss: 0.2812726744369869
# Epoch: 100%|██████████| 3/3 [1:35:35<00:00, 1911.70s/it]Validation Accuracy: 0.8702628530450132

# Macro F1 Score: 0.8176388332595512
# Accuracy score: 0.8937683771222612 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (552 > 512). Running this sequence through BERT will result in indexing errors
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 92247
# Seed value: 2019
# Train loss: 0.3684447507625636
# Epoch:  33%|███▎      | 1/3 [31:42<1:03:25, 1902.71s/it]Validation Accuracy: 0.8523968446601942
# Train loss: 0.37917312746011483
# Epoch:  67%|██████▋   | 2/3 [1:03:21<31:41, 1901.51s/it]Validation Accuracy: 0.859375
# Train loss: 0.39026388943071294
# Epoch: 100%|██████████| 3/3 [1:34:58<00:00, 1899.57s/it]Validation Accuracy: 0.8076594218887908

# Macro F1 Score: 0.46758459385877427
# Accuracy score: 0.8087830788200702 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (552 > 512). Running this sequence through BERT will result in indexing errors
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: English
# Sample Size: 92247
# Seed value: 2020
# Train loss: 0.3446399428046002
# Epoch:  33%|███▎      | 1/3 [31:41<1:03:22, 1901.17s/it]Validation Accuracy: 0.8644982899382171
# Train loss: 0.302568918340518
# Epoch:  67%|██████▋   | 2/3 [1:03:24<31:41, 1901.96s/it]Validation Accuracy: 0.8707248455428067
# Train loss: 0.3048158748304078
# Epoch: 100%|██████████| 3/3 [1:35:17<00:00, 1905.97s/it]Validation Accuracy: 0.8654843336275375

In [ ]:
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]Train loss: 0.34593701649986
# Epoch:  33%|███▎      | 1/3 [24:15<48:31, 1455.74s/it]Validation Accuracy: 0.8688038793103449
# Train loss: 0.2982849344799338
# Epoch:  67%|██████▋   | 2/3 [48:08<24:08, 1448.94s/it]Validation Accuracy: 0.8696120689655172
# Train loss: 0.2785517657614823
# Epoch: 100%|██████████| 3/3 [1:12:21<00:00, 1447.25s/it]Validation Accuracy: 0.8748204022988506


# Zero Shot Model Summary:
# Language: French 

# Macro F1 Score: 0.5487104810360981
# Accuracy score: 0.5860655737704918 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 16
# Seed value: 2018
# Train loss: 0.6053143739700317
# Epoch:  33%|███▎      | 1/3 [00:00<00:01,  1.54it/s]Validation Accuracy: 0.5868055555555556
# Train loss: 0.5899794101715088
# Epoch:  67%|██████▋   | 2/3 [00:01<00:00,  1.54it/s]Validation Accuracy: 0.6069444444444445
# Train loss: 0.4926532804965973
# Epoch: 100%|██████████| 3/3 [00:01<00:00,  1.54it/s]Validation Accuracy: 0.6430555555555555

# Macro F1 Score: 0.5437418317080527
# Accuracy score: 0.6188524590163934 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 16
# Seed value: 2019
# Train loss: 1.1067527532577515
# Epoch:  33%|███▎      | 1/3 [00:00<00:01,  1.52it/s]Validation Accuracy: 0.5652777777777778
# Train loss: 0.9282300472259521
# Epoch:  67%|██████▋   | 2/3 [00:01<00:00,  1.53it/s]Validation Accuracy: 0.5791666666666667
# Train loss: 0.9517414569854736
# Epoch: 100%|██████████| 3/3 [00:01<00:00,  1.54it/s]Validation Accuracy: 0.6027777777777777

# Macro F1 Score: 0.5473098330241187
# Accuracy score: 0.6188524590163934 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 16
# Seed value: 2020
# Train loss: 0.8084472417831421
# Epoch:  33%|███▎      | 1/3 [00:00<00:01,  1.57it/s]Validation Accuracy: 0.5840277777777778
# Train loss: 0.7122196555137634
# Epoch:  67%|██████▋   | 2/3 [00:01<00:00,  1.57it/s]Validation Accuracy: 0.5805555555555555
# Train loss: 0.8241579532623291
# Epoch: 100%|██████████| 3/3 [00:01<00:00,  1.56it/s]Validation Accuracy: 0.5979166666666667

# Macro F1 Score: 0.5191883718661594
# Accuracy score: 0.5860655737704918 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 16
# Seed value: 2021
# Train loss: 0.8900033831596375
# Epoch:  33%|███▎      | 1/3 [00:00<00:01,  1.53it/s]Validation Accuracy: 0.5833333333333333
# Train loss: 0.8552150130271912
# Epoch:  67%|██████▋   | 2/3 [00:01<00:00,  1.54it/s]Validation Accuracy: 0.6041666666666667
# Train loss: 0.8316570520401001
# Epoch: 100%|██████████| 3/3 [00:01<00:00,  1.55it/s]Validation Accuracy: 0.6069444444444445

# Macro F1 Score: 0.5597090052614584
# Accuracy score: 0.610655737704918 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 16
# Seed value: 2022
# Train loss: 0.6435708403587341
# Epoch:  33%|███▎      | 1/3 [00:00<00:01,  1.54it/s]Validation Accuracy: 0.5944444444444444
# Train loss: 0.6832331418991089
# Epoch:  67%|██████▋   | 2/3 [00:01<00:00,  1.54it/s]Validation Accuracy: 0.6118055555555555
# Train loss: 0.6912586688995361
# Epoch: 100%|██████████| 3/3 [00:01<00:00,  1.55it/s]Validation Accuracy: 0.6527777777777778

# Macro F1 Score: 0.6325002035333387
# Accuracy score: 0.6967213114754098 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 32
# Seed value: 2018
# Train loss: 0.8704765141010284
# Epoch:  33%|███▎      | 1/3 [00:00<00:01,  1.11it/s]Validation Accuracy: 0.5638888888888889
# Train loss: 0.8830522000789642
# Epoch:  67%|██████▋   | 2/3 [00:01<00:00,  1.13it/s]Validation Accuracy: 0.5694444444444444
# Train loss: 0.7046071887016296
# Epoch: 100%|██████████| 3/3 [00:02<00:00,  1.12it/s]Validation Accuracy: 0.6354166666666667

# Macro F1 Score: 0.5387943795404342
# Accuracy score: 0.6598360655737705 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 32
# Seed value: 2019
# Train loss: 0.7374086678028107
# Epoch:  33%|███▎      | 1/3 [00:00<00:01,  1.17it/s]Validation Accuracy: 0.6333333333333333
# Train loss: 0.6454640626907349
# Epoch:  67%|██████▋   | 2/3 [00:01<00:00,  1.16it/s]Validation Accuracy: 0.6465277777777778
# Train loss: 0.6088263094425201
# Epoch: 100%|██████████| 3/3 [00:02<00:00,  1.16it/s]Validation Accuracy: 0.6430555555555555

# Macro F1 Score: 0.5192794821277794
# Accuracy score: 0.6557377049180327 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 32
# Seed value: 2020
# Train loss: 0.7822281420230865
# Epoch:  33%|███▎      | 1/3 [00:00<00:01,  1.13it/s]Validation Accuracy: 0.6090277777777777
# Train loss: 0.7287075817584991
# Epoch:  67%|██████▋   | 2/3 [00:01<00:00,  1.12it/s]Validation Accuracy: 0.6055555555555555
# Train loss: 0.5211212784051895
# Epoch: 100%|██████████| 3/3 [00:02<00:00,  1.12it/s]Validation Accuracy: 0.6263888888888889

# Macro F1 Score: 0.6057496590074494
# Accuracy score: 0.6844262295081968 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 32
# Seed value: 2021
# Train loss: 0.8913013339042664
# Epoch:  33%|███▎      | 1/3 [00:00<00:01,  1.14it/s]Validation Accuracy: 0.6208333333333333
# Train loss: 0.5985569059848785
# Epoch:  67%|██████▋   | 2/3 [00:01<00:00,  1.14it/s]Validation Accuracy: 0.6555555555555556
# Train loss: 0.573823481798172
# Epoch: 100%|██████████| 3/3 [00:02<00:00,  1.14it/s]Validation Accuracy: 0.6666666666666667

# Macro F1 Score: 0.5188071188071188
# Accuracy score: 0.6639344262295082 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 32
# Seed value: 2022
# Train loss: 0.8934590518474579
# Epoch:  33%|███▎      | 1/3 [00:00<00:01,  1.16it/s]Validation Accuracy: 0.6625
# Train loss: 0.6475479602813721
# Epoch:  67%|██████▋   | 2/3 [00:01<00:00,  1.15it/s]Validation Accuracy: 0.6659722222222222
# Train loss: 0.5964289009571075
# Epoch: 100%|██████████| 3/3 [00:02<00:00,  1.15it/s]Validation Accuracy: 0.6916666666666667

# Macro F1 Score: 0.484272037796842
# Accuracy score: 0.6516393442622951 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 64
# Seed value: 2018
# Train loss: 0.8019826114177704
# Epoch:  33%|███▎      | 1/3 [00:01<00:02,  1.34s/it]Validation Accuracy: 0.6243055555555556
# Train loss: 0.660676084458828
# Epoch:  67%|██████▋   | 2/3 [00:02<00:01,  1.34s/it]Validation Accuracy: 0.6555555555555556
# Train loss: 0.5318491756916046
# Epoch: 100%|██████████| 3/3 [00:03<00:00,  1.33s/it]Validation Accuracy: 0.6458333333333333

# Macro F1 Score: 0.489272632129775
# Accuracy score: 0.6721311475409836 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 64
# Seed value: 2019
# Train loss: 0.6555497944355011
# Epoch:  33%|███▎      | 1/3 [00:01<00:02,  1.32s/it]Validation Accuracy: 0.6590277777777778
# Train loss: 0.5259348303079605
# Epoch:  67%|██████▋   | 2/3 [00:02<00:01,  1.32s/it]Validation Accuracy: 0.6777777777777778
# Train loss: 0.45486415550112724
# Epoch: 100%|██████████| 3/3 [00:03<00:00,  1.33s/it]Validation Accuracy: 0.6493055555555556

# Macro F1 Score: 0.5318136464809271
# Accuracy score: 0.6926229508196722 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 64
# Seed value: 2020
# Train loss: 0.6783757209777832
# Epoch:  33%|███▎      | 1/3 [00:01<00:02,  1.35s/it]Validation Accuracy: 0.6555555555555556
# Train loss: 0.5456586480140686
# Epoch:  67%|██████▋   | 2/3 [00:02<00:01,  1.34s/it]Validation Accuracy: 0.6868055555555556
# Train loss: 0.5005174353718758
# Epoch: 100%|██████████| 3/3 [00:03<00:00,  1.33s/it]Validation Accuracy: 0.6881944444444444

# Macro F1 Score: 0.5299812308604168
# Accuracy score: 0.680327868852459 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 64
# Seed value: 2021
# Train loss: 0.791699692606926
# Epoch:  33%|███▎      | 1/3 [00:01<00:02,  1.32s/it]Validation Accuracy: 0.6243055555555556
# Train loss: 0.5215376317501068
# Epoch:  67%|██████▋   | 2/3 [00:02<00:01,  1.32s/it]Validation Accuracy: 0.6743055555555555
# Train loss: 0.4719793051481247
# Epoch: 100%|██████████| 3/3 [00:03<00:00,  1.32s/it]Validation Accuracy: 0.6680555555555555

# Macro F1 Score: 0.4979545399530392
# Accuracy score: 0.6516393442622951 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 64
# Seed value: 2022
# Train loss: 0.8469303771853447
# Epoch:  33%|███▎      | 1/3 [00:01<00:02,  1.32s/it]Validation Accuracy: 0.695138888888889
# Train loss: 0.6987145394086838
# Epoch:  67%|██████▋   | 2/3 [00:02<00:01,  1.32s/it]Validation Accuracy: 0.6993055555555555
# Train loss: 0.6430374085903168
# Epoch: 100%|██████████| 3/3 [00:03<00:00,  1.32s/it]Validation Accuracy: 0.6854166666666667

# Macro F1 Score: 0.46198448346263776
# Accuracy score: 0.6680327868852459 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 128
# Seed value: 2018
# Train loss: 0.8640585541725159
# Epoch:  33%|███▎      | 1/3 [00:02<00:04,  2.26s/it]Validation Accuracy: 0.6826388888888889
# Train loss: 0.639813207089901
# Epoch:  67%|██████▋   | 2/3 [00:04<00:02,  2.27s/it]Validation Accuracy: 0.6618055555555555
# Train loss: 0.5732815340161324
# Epoch: 100%|██████████| 3/3 [00:06<00:00,  2.29s/it]Validation Accuracy: 0.6840277777777778

# Macro F1 Score: 0.4353812333680436
# Accuracy score: 0.6721311475409836 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 128
# Seed value: 2019
# Train loss: 0.7274994105100632
# Epoch:  33%|███▎      | 1/3 [00:02<00:04,  2.29s/it]Validation Accuracy: 0.6604166666666667
# Train loss: 0.5698614791035652
# Epoch:  67%|██████▋   | 2/3 [00:04<00:02,  2.27s/it]Validation Accuracy: 0.6729166666666667
# Train loss: 0.47679413110017776
# Epoch: 100%|██████████| 3/3 [00:06<00:00,  2.25s/it]Validation Accuracy: 0.6555555555555556

# Macro F1 Score: 0.6074490358126722
# Accuracy score: 0.7008196721311475 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 128
# Seed value: 2020
# Train loss: 0.77704643830657
# Epoch:  33%|███▎      | 1/3 [00:02<00:04,  2.23s/it]Validation Accuracy: 0.6888888888888889
# Train loss: 0.6182023957371712
# Epoch:  67%|██████▋   | 2/3 [00:04<00:02,  2.23s/it]Validation Accuracy: 0.6826388888888889
# Train loss: 0.5305736809968948
# Epoch: 100%|██████████| 3/3 [00:06<00:00,  2.23s/it]Validation Accuracy: 0.7152777777777778

# Macro F1 Score: 0.5813502424468482
# Accuracy score: 0.7172131147540983 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 128
# Seed value: 2021
# Train loss: 0.7752678506076336
# Epoch:  33%|███▎      | 1/3 [00:02<00:04,  2.35s/it]Validation Accuracy: 0.6805555555555556
# Train loss: 0.5674717091023922
# Epoch:  67%|██████▋   | 2/3 [00:04<00:02,  2.31s/it]Validation Accuracy: 0.6916666666666667
# Train loss: 0.5566453859210014
# Epoch: 100%|██████████| 3/3 [00:06<00:00,  2.29s/it]Validation Accuracy: 0.6930555555555555

# Macro F1 Score: 0.5339773576549396
# Accuracy score: 0.7172131147540983 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 128
# Seed value: 2022
# Train loss: 0.6996987015008926
# Epoch:  33%|███▎      | 1/3 [00:02<00:04,  2.26s/it]Validation Accuracy: 0.6541666666666667
# Train loss: 0.5598838143050671
# Epoch:  67%|██████▋   | 2/3 [00:04<00:02,  2.27s/it]Validation Accuracy: 0.6604166666666667
# Train loss: 0.48925821483135223
# Epoch: 100%|██████████| 3/3 [00:06<00:00,  2.30s/it]Validation Accuracy: 0.6680555555555555

# Macro F1 Score: 0.5846994535519126
# Accuracy score: 0.6885245901639344 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 256
# Seed value: 2018
# Train loss: 0.6906720548868179
# Epoch:  33%|███▎      | 1/3 [00:04<00:08,  4.04s/it]Validation Accuracy: 0.6680555555555555
# Train loss: 0.5906497836112976
# Epoch:  67%|██████▋   | 2/3 [00:08<00:04,  4.05s/it]Validation Accuracy: 0.6854166666666667
# Train loss: 0.5368122979998589
# Epoch: 100%|██████████| 3/3 [00:12<00:00,  4.08s/it]Validation Accuracy: 0.7229166666666667

# Macro F1 Score: 0.5732331411543694
# Accuracy score: 0.6680327868852459 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 256
# Seed value: 2019
# Train loss: 0.6890291515737772
# Epoch:  33%|███▎      | 1/3 [00:04<00:08,  4.20s/it]Validation Accuracy: 0.6743055555555555
# Train loss: 0.5768622383475304
# Epoch:  67%|██████▋   | 2/3 [00:08<00:04,  4.17s/it]Validation Accuracy: 0.6729166666666667
# Train loss: 0.5187756437808275
# Epoch: 100%|██████████| 3/3 [00:12<00:00,  4.15s/it]Validation Accuracy: 0.73125

# Macro F1 Score: 0.5847534677701365
# Accuracy score: 0.7008196721311475 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 256
# Seed value: 2020
# Train loss: 0.7282280921936035
# Epoch:  33%|███▎      | 1/3 [00:04<00:08,  4.02s/it]Validation Accuracy: 0.6993055555555555
# Train loss: 0.5838286560028791
# Epoch:  67%|██████▋   | 2/3 [00:08<00:04,  4.02s/it]Validation Accuracy: 0.6930555555555555
# Train loss: 0.5097991451621056
# Epoch: 100%|██████████| 3/3 [00:12<00:00,  4.05s/it]Validation Accuracy: 0.6569444444444444

# Macro F1 Score: 0.6074490358126722
# Accuracy score: 0.7008196721311475 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 256
# Seed value: 2021
# Train loss: 0.6603494454175234
# Epoch:  33%|███▎      | 1/3 [00:04<00:08,  4.06s/it]Validation Accuracy: 0.6541666666666667
# Train loss: 0.559552313759923
# Epoch:  67%|██████▋   | 2/3 [00:08<00:04,  4.08s/it]Validation Accuracy: 0.6569444444444444
# Train loss: 0.48664303310215473
# Epoch: 100%|██████████| 3/3 [00:12<00:00,  4.08s/it]Validation Accuracy: 0.6208333333333333

# Macro F1 Score: 0.6572440202436163
# Accuracy score: 0.7008196721311475 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 256
# Seed value: 2022
# Train loss: 0.682473411783576
# Epoch:  33%|███▎      | 1/3 [00:04<00:08,  4.04s/it]Validation Accuracy: 0.6618055555555555
# Train loss: 0.559933740645647
# Epoch:  67%|██████▋   | 2/3 [00:08<00:04,  4.04s/it]Validation Accuracy: 0.6840277777777778
# Train loss: 0.5080154798924923
# Epoch: 100%|██████████| 3/3 [00:12<00:00,  4.08s/it]Validation Accuracy: 0.6618055555555555

# Macro F1 Score: 0.6911392405063291
# Accuracy score: 0.7295081967213115 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 1067
# Seed value: 2018
# Train loss: 0.629720612248378
# Epoch:  33%|███▎      | 1/3 [00:15<00:31, 15.55s/it]Validation Accuracy: 0.7041666666666667
# Train loss: 0.5448412912995068
# Epoch:  67%|██████▋   | 2/3 [00:31<00:15, 15.53s/it]Validation Accuracy: 0.7055555555555555
# Train loss: 0.4813313753302418
# Epoch: 100%|██████████| 3/3 [00:46<00:00, 15.61s/it]Validation Accuracy: 0.7340277777777777

# Macro F1 Score: 0.8197371490784664
# Accuracy score: 0.8442622950819673 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 1067
# Seed value: 2019
# Train loss: 0.6498969575362419
# Epoch:  33%|███▎      | 1/3 [00:15<00:31, 15.77s/it]Validation Accuracy: 0.7090277777777778
# Train loss: 0.5573823887910416
# Epoch:  67%|██████▋   | 2/3 [00:31<00:15, 15.78s/it]Validation Accuracy: 0.7027777777777777
# Train loss: 0.47990676204660043
# Epoch: 100%|██████████| 3/3 [00:47<00:00, 15.80s/it]Validation Accuracy: 0.7076388888888889

# Macro F1 Score: 0.806957897662882
# Accuracy score: 0.8483606557377049 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 1067
# Seed value: 2020
# Train loss: 0.6199074031701729
# Epoch:  33%|███▎      | 1/3 [00:15<00:31, 15.67s/it]Validation Accuracy: 0.6777777777777778
# Train loss: 0.5361942172940097
# Epoch:  67%|██████▋   | 2/3 [00:31<00:15, 15.75s/it]Validation Accuracy: 0.6340277777777777
# Train loss: 0.4595312538431652
# Epoch: 100%|██████████| 3/3 [00:47<00:00, 15.75s/it]Validation Accuracy: 0.6590277777777778

# Macro F1 Score: 0.8662280701754386
# Accuracy score: 0.8852459016393442 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 1067
# Seed value: 2021
# Train loss: 0.6301538681806024
# Epoch:  33%|███▎      | 1/3 [00:15<00:31, 15.59s/it]Validation Accuracy: 0.7201388888888889
# Train loss: 0.5482929566902901
# Epoch:  67%|██████▋   | 2/3 [00:31<00:15, 15.57s/it]Validation Accuracy: 0.7277777777777777
# Train loss: 0.4588816266006498
# Epoch: 100%|██████████| 3/3 [00:46<00:00, 15.57s/it]Validation Accuracy: 0.6916666666666667

# Macro F1 Score: 0.7860212224853108
# Accuracy score: 0.8360655737704918 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: French
# Sample Size: 1067
# Seed value: 2022
# Train loss: 0.6259974459213997
# Epoch:  33%|███▎      | 1/3 [00:15<00:31, 15.84s/it]Validation Accuracy: 0.6965277777777777
# Train loss: 0.5515195122405664
# Epoch:  67%|██████▋   | 2/3 [00:31<00:15, 15.80s/it]Validation Accuracy: 0.726388888888889
# Train loss: 0.4877439870763181
# Epoch: 100%|██████████| 3/3 [00:47<00:00, 15.76s/it]Validation Accuracy: 0.7201388888888889

# Macro F1 Score: 0.622680412371134
# Accuracy score: 0.7540983606557377

In [ ]:
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]Train loss: 0.47433087877913066
# Epoch:  33%|███▎      | 1/3 [23:10<46:20, 1390.03s/it]Validation Accuracy: 0.8245877061469266
# Train loss: 0.4936049662121814
# Epoch:  67%|██████▋   | 2/3 [46:23<23:11, 1391.00s/it]Validation Accuracy: 0.8053473263368315
# Train loss: 0.49543079081068914
# Epoch: 100%|██████████| 3/3 [1:09:33<00:00, 1391.18s/it]Validation Accuracy: 0.8054202065633851


# Zero Shot Model Summary:
# Language: Arabic 

# Macro F1 Score: 0.4409349946875302
# Accuracy score: 0.7887007601935038 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 16
# Seed value: 2018
# Train loss: 0.47678956389427185
# Epoch:  33%|███▎      | 1/3 [00:02<00:04,  2.18s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.4873213768005371
# Epoch:  67%|██████▋   | 2/3 [00:04<00:02,  2.18s/it]Validation Accuracy: 0.7839673913043478
# Train loss: 0.4843035340309143
# Epoch: 100%|██████████| 3/3 [00:06<00:00,  2.18s/it]Validation Accuracy: 0.7880434782608695

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 16
# Seed value: 2019
# Train loss: 0.47897496819496155
# Epoch:  33%|███▎      | 1/3 [00:02<00:04,  2.16s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.4883110523223877
# Epoch:  67%|██████▋   | 2/3 [00:04<00:02,  2.15s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.48860523104667664
# Epoch: 100%|██████████| 3/3 [00:06<00:00,  2.15s/it]Validation Accuracy: 0.7839673913043478

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 16
# Seed value: 2020
# Train loss: 0.4766619801521301
# Epoch:  33%|███▎      | 1/3 [00:02<00:04,  2.15s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.4729042649269104
# Epoch:  67%|██████▋   | 2/3 [00:04<00:02,  2.15s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.47194594144821167
# Epoch: 100%|██████████| 3/3 [00:06<00:00,  2.15s/it]Validation Accuracy: 0.7839673913043478

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 16
# Seed value: 2021
# Train loss: 0.4793074429035187
# Epoch:  33%|███▎      | 1/3 [00:02<00:04,  2.16s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.46476367115974426
# Epoch:  67%|██████▋   | 2/3 [00:04<00:02,  2.16s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.49083223938941956
# Epoch: 100%|██████████| 3/3 [00:06<00:00,  2.15s/it]Validation Accuracy: 0.7921195652173914

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 16
# Seed value: 2022
# Train loss: 0.48052307963371277
# Epoch:  33%|███▎      | 1/3 [00:02<00:04,  2.15s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.4745211899280548
# Epoch:  67%|██████▋   | 2/3 [00:04<00:02,  2.15s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.48432403802871704
# Epoch: 100%|██████████| 3/3 [00:06<00:00,  2.15s/it]Validation Accuracy: 0.7880434782608695

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 32
# Seed value: 2018
# Train loss: 0.5362682342529297
# Epoch:  33%|███▎      | 1/3 [00:02<00:04,  2.40s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5344464927911758
# Epoch:  67%|██████▋   | 2/3 [00:04<00:02,  2.40s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5338150411844254
# Epoch: 100%|██████████| 3/3 [00:07<00:00,  2.40s/it]Validation Accuracy: 0.7880434782608695

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 32
# Seed value: 2019
# Train loss: 0.5382117182016373
# Epoch:  33%|███▎      | 1/3 [00:02<00:04,  2.37s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5405432879924774
# Epoch:  67%|██████▋   | 2/3 [00:04<00:02,  2.37s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5373820811510086
# Epoch: 100%|██████████| 3/3 [00:07<00:00,  2.37s/it]Validation Accuracy: 0.7880434782608695

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 32
# Seed value: 2020
# Train loss: 0.5340271294116974
# Epoch:  33%|███▎      | 1/3 [00:02<00:04,  2.38s/it]Validation Accuracy: 0.779891304347826
# Train loss: 0.5399025082588196
# Epoch:  67%|██████▋   | 2/3 [00:04<00:02,  2.39s/it]Validation Accuracy: 0.7839673913043478
# Train loss: 0.5367260575294495
# Epoch: 100%|██████████| 3/3 [00:07<00:00,  2.39s/it]Validation Accuracy: 0.7880434782608695

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 32
# Seed value: 2021
# Train loss: 0.5339076668024063
# Epoch:  33%|███▎      | 1/3 [00:02<00:04,  2.36s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5589636415243149
# Epoch:  67%|██████▋   | 2/3 [00:04<00:02,  2.36s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5320451706647873
# Epoch: 100%|██████████| 3/3 [00:07<00:00,  2.37s/it]Validation Accuracy: 0.7839673913043478

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 32
# Seed value: 2022
# Train loss: 0.5334031134843826
# Epoch:  33%|███▎      | 1/3 [00:02<00:04,  2.37s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5346644967794418
# Epoch:  67%|██████▋   | 2/3 [00:04<00:02,  2.37s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5289856940507889
# Epoch: 100%|██████████| 3/3 [00:07<00:00,  2.37s/it]Validation Accuracy: 0.7880434782608695

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 64
# Seed value: 2018
# Train loss: 0.5422306954860687
# Epoch:  33%|███▎      | 1/3 [00:02<00:05,  2.82s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5301393046975136
# Epoch:  67%|██████▋   | 2/3 [00:05<00:02,  2.82s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5301894992589951
# Epoch: 100%|██████████| 3/3 [00:08<00:00,  2.83s/it]Validation Accuracy: 0.7880434782608695

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 64
# Seed value: 2019
# Train loss: 0.537293404340744
# Epoch:  33%|███▎      | 1/3 [00:02<00:05,  2.84s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5381098836660385
# Epoch:  67%|██████▋   | 2/3 [00:05<00:02,  2.84s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5327314510941505
# Epoch: 100%|██████████| 3/3 [00:08<00:00,  2.84s/it]Validation Accuracy: 0.7839673913043478

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 64
# Seed value: 2020
# Train loss: 0.5256571769714355
# Epoch:  33%|███▎      | 1/3 [00:02<00:05,  2.82s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5317957550287247
# Epoch:  67%|██████▋   | 2/3 [00:05<00:02,  2.83s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5357784256339073
# Epoch: 100%|██████████| 3/3 [00:08<00:00,  2.84s/it]Validation Accuracy: 0.7880434782608695

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 64
# Seed value: 2021
# Train loss: 0.5289135947823524
# Epoch:  33%|███▎      | 1/3 [00:02<00:05,  2.83s/it]Validation Accuracy: 0.7839673913043478
# Train loss: 0.5491465330123901
# Epoch:  67%|██████▋   | 2/3 [00:05<00:02,  2.82s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5295693501830101
# Epoch: 100%|██████████| 3/3 [00:08<00:00,  2.84s/it]Validation Accuracy: 0.7921195652173914

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 64
# Seed value: 2022
# Train loss: 0.540005512535572
# Epoch:  33%|███▎      | 1/3 [00:02<00:05,  2.82s/it]Validation Accuracy: 0.779891304347826
# Train loss: 0.5389735698699951
# Epoch:  67%|██████▋   | 2/3 [00:05<00:02,  2.82s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5260805413126945
# Epoch: 100%|██████████| 3/3 [00:08<00:00,  2.85s/it]Validation Accuracy: 0.7921195652173914

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 128
# Seed value: 2018
# Train loss: 0.5286239385604858
# Epoch:  33%|███▎      | 1/3 [00:03<00:07,  3.75s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5196566991508007
# Epoch:  67%|██████▋   | 2/3 [00:07<00:03,  3.74s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5184898786246777
# Epoch: 100%|██████████| 3/3 [00:11<00:00,  3.73s/it]Validation Accuracy: 0.7921195652173914

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 128
# Seed value: 2019
# Train loss: 0.5198039207607508
# Epoch:  33%|███▎      | 1/3 [00:03<00:07,  3.71s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5253091603517532
# Epoch:  67%|██████▋   | 2/3 [00:07<00:03,  3.71s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5158082693815231
# Epoch: 100%|██████████| 3/3 [00:11<00:00,  3.71s/it]Validation Accuracy: 0.7839673913043478

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 128
# Seed value: 2020
# Train loss: 0.5200199373066425
# Epoch:  33%|███▎      | 1/3 [00:03<00:07,  3.73s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5229551363736391
# Epoch:  67%|██████▋   | 2/3 [00:07<00:03,  3.77s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5271130166947842
# Epoch: 100%|██████████| 3/3 [00:11<00:00,  3.78s/it]Validation Accuracy: 0.7921195652173914

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 128
# Seed value: 2021
# Train loss: 0.527061078697443
# Epoch:  33%|███▎      | 1/3 [00:03<00:07,  3.71s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5194373205304146
# Epoch:  67%|██████▋   | 2/3 [00:07<00:03,  3.72s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5242746062576771
# Epoch: 100%|██████████| 3/3 [00:11<00:00,  3.72s/it]Validation Accuracy: 0.7921195652173914

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 128
# Seed value: 2022
# Train loss: 0.5266167521476746
# Epoch:  33%|███▎      | 1/3 [00:03<00:07,  3.78s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5174127370119095
# Epoch:  67%|██████▋   | 2/3 [00:07<00:03,  3.76s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5238984115421772
# Epoch: 100%|██████████| 3/3 [00:11<00:00,  3.74s/it]Validation Accuracy: 0.7880434782608695

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 256
# Seed value: 2018
# Train loss: 0.5246865507215261
# Epoch:  33%|███▎      | 1/3 [00:05<00:11,  5.50s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5209596157073975
# Epoch:  67%|██████▋   | 2/3 [00:11<00:05,  5.51s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5191416069865227
# Epoch: 100%|██████████| 3/3 [00:16<00:00,  5.50s/it]Validation Accuracy: 0.7839673913043478

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 256
# Seed value: 2019
# Train loss: 0.5228016106411815
# Epoch:  33%|███▎      | 1/3 [00:05<00:11,  5.53s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5137968696653843
# Epoch:  67%|██████▋   | 2/3 [00:11<00:05,  5.52s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5169773977249861
# Epoch: 100%|██████████| 3/3 [00:16<00:00,  5.55s/it]Validation Accuracy: 0.7921195652173914

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 256
# Seed value: 2020
# Train loss: 0.5220542345196009
# Epoch:  33%|███▎      | 1/3 [00:05<00:11,  5.53s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5160006675869226
# Epoch:  67%|██████▋   | 2/3 [00:11<00:05,  5.57s/it]Validation Accuracy: 0.7839673913043478
# Train loss: 0.5233323741704226
# Epoch: 100%|██████████| 3/3 [00:16<00:00,  5.59s/it]Validation Accuracy: 0.7921195652173914

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 256
# Seed value: 2021
# Train loss: 0.5195233915001154
# Epoch:  33%|███▎      | 1/3 [00:05<00:10,  5.50s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5157914254814386
# Epoch:  67%|██████▋   | 2/3 [00:11<00:05,  5.53s/it]Validation Accuracy: 0.7839673913043478
# Train loss: 0.5169495511800051
# Epoch: 100%|██████████| 3/3 [00:16<00:00,  5.56s/it]Validation Accuracy: 0.7921195652173914

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 256
# Seed value: 2022
# Train loss: 0.5234786663204432
# Epoch:  33%|███▎      | 1/3 [00:05<00:11,  5.59s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5190018750727177
# Epoch:  67%|██████▋   | 2/3 [00:11<00:05,  5.57s/it]Validation Accuracy: 0.7839673913043478
# Train loss: 0.5175019651651382
# Epoch: 100%|██████████| 3/3 [00:16<00:00,  5.57s/it]Validation Accuracy: 0.7880434782608695

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 5064
# Seed value: 2018
# Train loss: 0.5172838105200217
# Epoch:  33%|███▎      | 1/3 [01:13<02:27, 73.96s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.519950428407651
# Epoch:  67%|██████▋   | 2/3 [02:28<01:14, 74.01s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5187662234911783
# Epoch: 100%|██████████| 3/3 [03:42<00:00, 74.27s/it]Validation Accuracy: 0.7880434782608695

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 5064
# Seed value: 2019
# Train loss: 0.5166931613480631
# Epoch:  33%|███▎      | 1/3 [01:14<02:28, 74.44s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5179957420762029
# Epoch:  67%|██████▋   | 2/3 [02:28<01:14, 74.38s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5183741434327436
# Epoch: 100%|██████████| 3/3 [03:42<00:00, 74.31s/it]Validation Accuracy: 0.7921195652173914

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 5064
# Seed value: 2020
# Train loss: 0.518989954367045
# Epoch:  33%|███▎      | 1/3 [01:14<02:28, 74.50s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5180425098171746
# Epoch:  67%|██████▋   | 2/3 [02:28<01:14, 74.39s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5176849400959557
# Epoch: 100%|██████████| 3/3 [03:42<00:00, 74.20s/it]Validation Accuracy: 0.7880434782608695

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 5064
# Seed value: 2021
# Train loss: 0.5192183481782017
# Epoch:  33%|███▎      | 1/3 [01:14<02:28, 74.08s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5182036432181999
# Epoch:  67%|██████▋   | 2/3 [02:28<01:14, 74.23s/it]Validation Accuracy: 0.779891304347826
# Train loss: 0.5187887173815856
# Epoch: 100%|██████████| 3/3 [03:42<00:00, 74.14s/it]Validation Accuracy: 0.7921195652173914

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547 

# ====================================================================================================
# t_total value of -1 results in schedule not being applied
# Epoch:   0%|          | 0/3 [00:00<?, ?it/s]
# Model Summary:
# Language: Arabic
# Sample Size: 5064
# Seed value: 2022
# Train loss: 0.5191312888255255
# Epoch:  33%|███▎      | 1/3 [01:13<02:27, 73.85s/it]Validation Accuracy: 0.7880434782608695
# Train loss: 0.5192586619301173
# Epoch:  67%|██████▋   | 2/3 [02:28<01:13, 74.00s/it]Validation Accuracy: 0.7921195652173914
# Train loss: 0.5185295956838019
# Epoch: 100%|██████████| 3/3 [03:42<00:00, 74.08s/it]Validation Accuracy: 0.7921195652173914

# Macro F1 Score: 0.44084983099951713
# Accuracy score: 0.7884283246977547